# Expermenting with Conditional Random Field Model
-------------

Loading data and models for CRF cross validations

In [1]:
import __init__
import models.crf as crf
import models.cross_validator as cross_validator
import models.datahandler as datahandler
from models.datamodel import DataModel
import models.outputmaker as outputmaker
import evaluator.metrics as metrics

trial = datahandler.load_train('../data/dataset/tsd_trial.csv', verbose=True)
train = datahandler.load_train('../data/dataset/tsd_train.csv', verbose=True)

/home/hb/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


Initialize `CRF Model` Methods

In [2]:
datamodel = DataModel(model='crf')

output_maker = outputmaker.crf_output

evaluator = metrics

In [3]:
X_train, y_train , train_taboo_words, train_texts = cross_validator.load_data(train, [i for i in range(train.shape[0])], 
                                                    datamodel, logger=True)

X_test, y_test , test_taboo_words, test_texts = cross_validator.load_data(trial, [i for i in range(trial.shape[0])], 
                                                           datamodel, logger=True)

100%|██████████| 690/690 [00:50<00:00, 13.66it/s]


In [5]:
X_test[0]

[('Because', 'IN', 'normal', [0, 1, 2, 3, 4, 5, 6]),
 ('he', 'PRP', 'normal', [8, 9]),
 ('a', 'DT', 'normal', [13]),
 ('moron', 'NN', 'toxic', [15, 16, 17, 18, 19]),
 ('and', 'CC', 'normal', [21, 22, 23]),
 ('a', 'DT', 'normal', [25]),
 ('bigot', 'NNP', 'toxic', [27, 28, 29, 30, 31]),
 ('It', 'PRP', 'normal', [34, 35]),
 ('not', 'RB', 'normal', [39, 40, 41]),
 ('any', 'DT', 'normal', [43, 44, 45]),
 ('more', 'RBR', 'normal', [47, 48, 49, 50]),
 ('complicated', 'JJ', 'normal', [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]),
 ('than', 'IN', 'normal', [64, 65, 66, 67]),
 ('that', 'DT', 'normal', [69, 70, 71, 72])]

In [6]:
from tqdm import tqdm 
from nltk.stem.porter import *
stemmer = PorterStemmer()


special_chars = ['O', '4', '͞', 'ü', '😞', '»', '”', '\x7f', '"', '7', '*', '8', ')',
      '😂', '💀', '{', '0', 'ʻ', '💨', '•', '#', '_', '😊', '😜', 'ó', '😅', 
      '¬', '☠', '🙄', '😉', '😆', '―','(', '.', '?', '😵', '💥', ':', '🆘',
      '$', '=', '+', ';', '🔥', '😁','`', 'ï', '2', 'ê', 'é', '3', '-', '🤥',
      '⚽', 'É', '️', '!', '5', '✭', '9', '😈', 'ˈ', '😬', '@', '«', '/', '▀', '’',
      '😡', '1', '%', ',', '&', '6', '\\', ']', "'", '[', '^', '}', '⚾', '\u200b',
       '☭', '☹', '<', 'l', '™', '👎']


F_cleaner = lambda x: ''.join([w for w in x if w not in special_chars ]).lower()

     
def cleaning(X):
    cleaned_x = []
    for words in tqdm(X):
        cleaned_words = []
        for word in words :
            w, pos, tag, span = word
            cleaned_w = F_cleaner(w)
            if len(cleaned_w) != 0:
                #cleaned_words.append((cleaned_w, pos, tag, span))
                cleaned_words.append((stemmer.stem(cleaned_w), pos, tag, span) )
        cleaned_x.append(cleaned_words)
    return cleaned_x

In [7]:
X_train_clean = cleaning(X_train)
X_test_clean = cleaning(X_test)

100%|██████████| 690/690 [00:00<00:00, 1220.79it/s]


In [8]:
X_test_clean[0]

[('becaus', 'IN', 'normal', [0, 1, 2, 3, 4, 5, 6]),
 ('he', 'PRP', 'normal', [8, 9]),
 ('a', 'DT', 'normal', [13]),
 ('moron', 'NN', 'toxic', [15, 16, 17, 18, 19]),
 ('and', 'CC', 'normal', [21, 22, 23]),
 ('a', 'DT', 'normal', [25]),
 ('bigot', 'NNP', 'toxic', [27, 28, 29, 30, 31]),
 ('it', 'PRP', 'normal', [34, 35]),
 ('not', 'RB', 'normal', [39, 40, 41]),
 ('ani', 'DT', 'normal', [43, 44, 45]),
 ('more', 'RBR', 'normal', [47, 48, 49, 50]),
 ('compic', 'JJ', 'normal', [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]),
 ('than', 'IN', 'normal', [64, 65, 66, 67]),
 ('that', 'DT', 'normal', [69, 70, 71, 72])]

# Bayes Theorem
<img src="images/bayes-theorem.webp" style="width:400px;height:150px;">

* P(A|B) – the probability of event A occurring, given event B has occurred
* P(B|A) – the probability of event B occurring, given event A has occurred
* P(A) – the probability of event A
* P(B) – the probability of event B

<img src="images/bayes-theorem2.webp" style="width:500px;height:150px;">

* P(B|A–) – the probability of event B occurring given that event A– has occurred
* P(B|A+) – the probability of event B occurring given that event A+ has occurred


`In the special case above, events A– and A+ are mutually exclusive outcomes of event A.`

In [ ]:
word_matrix = set()
word2index = {}
index2word = {}